In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
texasCasesDataURL = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyCaseCountData.xlsx"
texasDeathsDataURL = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyFatalityCountData.xlsx"
texasTestingLegacyDataUrl = "https://dshs.texas.gov/coronavirus/TexasCOVID-19CumulativeTestsOverTimebyCounty.xlsx"
texasTestingDataUrl = "https://dshs.texas.gov/coronavirus/TexasCOVID-19CumulativeTestsbyCounty.xlsx"

In [ ]:
casesDf = pd.read_excel(texasCasesDataURL, skiprows=2, skipfooter=11)
deathsDf = pd.read_excel(texasDeathsDataURL, skiprows=2, skipfooter=2)
legacytestsDf = pd.read_excel(texasTestingLegacyDataUrl, skiprows=1, skipfooter=11)
testscasesDf = pd.read_excel(texasTestingDataUrl, skiprows=1, skipfooter=3)

In [ ]:
headers = casesDf.columns
headers = [i.split(' ')[1] for i in headers]
headers = ['2020-' + i for i in headers[1:]]
headers.insert(0, 'County')
casesDf.columns = headers

In [ ]:
casesDfMelted = casesDf.melt(id_vars='County', var_name='Date', value_name='Cases')
casesDfMelted.Date = pd.to_datetime(casesDfMelted.Date)

In [ ]:
headers = deathsDf.columns
headers = [i.split(' ')[1] for i in headers]
headers = ['2020-' + i for i in headers[1:]]
headers.insert(0, 'County')
deathsDf.columns = headers

In [ ]:
deathsDfMelted = deathsDf.melt(id_vars='County', var_name='Date', value_name='Deaths')
deathsDfMelted.Date = pd.to_datetime(casesDfMelted.Date)
deathsDfMelted.County = deathsDfMelted.County.str.capitalize()

In [ ]:
headers = legacytestsDf.columns
headers = [i.split(' ')[2:4] for i in headers]
headers = [' '.join(i) for i in headers]
headers = [i + ', 2020' for i in headers]
headers = [i.replace('*', '') for i in headers]
headers = [pd.to_datetime(i) for i in headers[1:]]
headers = [i.strftime("%Y-%m-%d") for i in headers]
headers.insert(0, 'County')
legacytestsDf.columns = headers

In [ ]:
headers = testscasesDf.columns
headers = [i.strftime("%Y-%m-%d") for i in headers[1:]]
headers.insert(0, 'County')
testscasesDf.columns = headers

In [ ]:
testsDf = pd.concat([legacytestsDf, testscasesDf], axis=1)
testsDf = testsDf.loc[:,~testsDf.columns.duplicated()]
testsDf.drop(columns=['2020-05-05', '2020-05-23', '2020-06-10'], inplace=True)

In [ ]:
testsDfMelted = testsDf.melt(id_vars='County', var_name='Date', value_name='Tests')
testsDfMelted.Date = pd.to_datetime(testsDfMelted.Date)

In [ ]:
unifiedTexasDf = pd.merge(testsDfMelted, casesDfMelted)
unifiedTexasDf = pd.merge(unifiedTexasDf, deathsDfMelted)
unifiedTexasDf.Tests = unifiedTexasDf.Tests.astype('int')

In [ ]:
graphDf = unifiedTexasDf.copy()
graphDf.set_index('Date', inplace=True)
graphDf.groupby('County')['Tests'].plot(legend=True)